
# Max overlapping timeseries algorithm

This algorithm has been designed to find the number of timeseries which has a minimum number of samples at the begining of each timeseries. 

For example, let's assume that we have timeseries produced by some type of meter with some periodic values. The start and end date of the series can be different for each file. To generate a set to apply a forecasting model using different input variables, we need to obtain a set of overlapping timeseries with a minimal duration to make sure the ML algorithm will capture the periodicity of the samples. The periods will depend on the type of problem we are studying. 

The proposed algorithm allows to know, for each date on, how many overlapping days of timeseries we have available. 

Let's assume that we have N meters which generates N files, each one including a timeseries with starts with a `start date` and finishes with an `end date`.

We must generate a dataframe containing:

|  `Start date` | `Timeseries ID` | `Duration of timeseries in days` = `End date` - `Start date` |

For example:

```
#Start date, meter ID, Duration in days
2019-06-02T09:37:00.000Z, vgbiwenoi2323, 367
2019-06-05T09:37:00.000Z, dscafweee3498, 450
2019-06-06T09:37:00.000Z, cncinnenr7325, 348
2019-06-09T09:37:00.000Z, onjdqweni8623, 317
2019-06-10T09:37:00.000Z, eiwhdoqwu3764, 347
2019-06-11T09:37:00.000Z, lidscbnqo1387, 227
2019-06-13T09:37:00.000Z, vgbiweeef2424, 367
2019-06-13T09:37:00.000Z, ebnqiunin1298, 387
2019-06-18T09:37:00.000Z, ommfiunun3546, 357
2019-06-21T09:37:00.000Z, tyrfeunht6543, 398
```



In [1]:
import pandas as pd
# Display all the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 40)

In [4]:
def run_algorithm(input_csv_filename, min_period):
    timeseries_df= pd.read_csv(input_csv_filename)
    timeseries_df['start'] = pd.to_datetime(timeseries_df['start'])

    # The dataframe must be ordered by increasing date
    timeseries_df=timeseries_df.sort_values(by="start")
    # We remove the index to make sure the loop follow the rows ordered by increasing date
    timeseries_df=timeseries_df.reset_index(drop=True)

    # We calculate the delta between rows and leave it in a column as integers
    timeseries_df['delta']= timeseries_df['start'].diff()
    timeseries_df.iloc[0, timeseries_df.columns.get_loc('delta')] = pd.Timedelta('0 days')
    timeseries_df['delta'] = timeseries_df['delta'].dt.days.astype('int64')

    # This new column stores  number of complete periods in the rows processed until the 
    # date of the current iteration
    timeseries_df['complete_periods']=0
    # this column stores the remaing days when the max of complete periods is reached 
    timeseries_df['duration_at_max']=0

    # Iterate all the rows to find the max value for complete_periods
    # Skipping first iteration
    
    max_complete_periods=0
    max_complete_periods_date=''
    for i in range(1,len(timeseries_df)) :
        #print('Iteration: ' + str(i))
        #timeseries_df.loc[0:i-1,'duration']=timeseries_df.loc[0:i-1,'duration']-timeseries_df.loc[i,'delta']
        timeseries_df.loc[0:i-1,'duration']-=timeseries_df.loc[i,'delta']
        timeseries_df.loc[i,'complete_periods'] = (timeseries_df.loc[0:i,'duration']>=min_period).sum()
        if (timeseries_df.loc[i,'complete_periods']>max_complete_periods):
            max_complete_periods=timeseries_df.loc[i,'complete_periods']
            max_complete_periods_date=timeseries_df.loc[i,'start']
            timeseries_df['duration_at_max']=timeseries_df['duration']

        #print('Number of completes periods')
        #print((timeseries_df.loc[0:i,'Duration']>=min_period).sum())
        # print(timeseries_df.loc[0:i,:])
        #print(timeseries_df.iloc[i, 0], timeseries_df.iloc[i, 2])

    return timeseries_df, max_complete_periods, max_complete_periods_date


In [8]:
input_csv_filename = 'sample_csv_files/time_ranges.csv'
min_period=360
timeseries_df = pd.read_csv(input_csv_filename)
#print(timeseries_df)

%timeit
#returned_df, max_complete_periods, max_complete_periods_date = run_algorithm(input_csv_filename, min_period)
print("Max complete periods: " + str(max_complete_periods))
#print(returned_df)

Max complete periods: 15655


In [9]:
print(returned_df.tail())
#print(returned_df.max())
#print(returned_df[returned_df['complete_periods']==returned_df['complete_periods'].max()])
print(returned_df['meterId'][returned_df['duration_at_max']>min_period]) 

                    start        meterId  duration  delta  complete_periods  \
16143 2021-03-04 23:00:00  SAG0205909972         3      3                 0   
16144 2021-03-05 23:00:00  SAG0205909973         3      1                 0   
16145 2021-03-06 23:00:00  SAG0205910047         3      1                 0   
16146 2021-03-08 01:00:00  SAG0205910036         0      1                 0   
16147 2021-03-09 23:00:00  SAG0205910046         1      1                 0   

       duration_at_max  
16143                7  
16144                6  
16145                5  
16146                1  
16147                1  
0        CIR0141449180
1        CIR0141600959
2        CIR0141601720
3        CIR0141682188
4        CIR0141441118
5        CIR0501631509
6        CIR0141691244
7        CIR0141692030
9        CIR0141600370
10       CIR0141448720
11       CIR0141456545
12       CIR0141606249
13       CIR0141449251
14       CIR0141601742
15       CIR0141601723
16       CIR0141601476
17     

### Find the best time window

In [10]:
csv_input_filename = 'sample_csv_files/time_ranges.csv'
timeseries_df = pd.read_csv(csv_input_filename)

# get longest period
max_duration= timeseries_df['duration'].max()
print('Max duration: ' + str(max_duration))

for min_period in range(360, max_duration, 5):
    returned_df, max_complete_periods, max_complete_periods_date = run_algorithm(csv_input_filename, min_period)
    #number_meters = returned_df[returned_df['duration_at_max'] > min_period].shape[0]
    print(f"Period of {min_period} days: {max_complete_periods} meters starting at {max_complete_periods_date}")

Max duration: 656
Period of 360 days: 15701 meters starting at 2020-03-04 23:00:00
Period of 365 days: 15699 meters starting at 2020-02-28 23:00:00
Period of 370 days: 15698 meters starting at 2020-03-01 23:00:00
Period of 375 days: 15695 meters starting at 2020-02-28 23:00:00
Period of 380 days: 15691 meters starting at 2020-02-23 23:00:00
Period of 385 days: 15688 meters starting at 2020-02-18 23:00:00
Period of 390 days: 15680 meters starting at 2020-02-15 23:00:00
Period of 395 days: 15677 meters starting at 2020-02-08 23:00:00
Period of 400 days: 15675 meters starting at 2020-02-05 23:00:00
Period of 405 days: 15669 meters starting at 2020-01-28 23:00:00
Period of 410 days: 15666 meters starting at 2020-01-25 01:00:00
Period of 415 days: 15663 meters starting at 2020-01-18 23:00:00
Period of 420 days: 15661 meters starting at 2020-01-14 14:00:00
Period of 425 days: 15657 meters starting at 2019-11-28 01:00:00
Period of 430 days: 15656 meters starting at 2019-11-28 01:00:00
Period 

### Run the algorithm with the best time window
We consider 590 days/15116 meters as the optimal value. We have to get the list of meters which match in this time window.

In [23]:
best_time_period = 590
results, max_complete_periods, max_complete_periods_date = run_algorithm(csv_input_filename, best_time_period)
meters = results[results['duration_at_max']>=best_time_period]['meterId']
print('The list has ' + str(meters.len()) + 'meters.')
print(meters)

AttributeError: 'Series' object has no attribute 'len'

In [22]:
meters = results[results['duration_at_max']>=best_time_period]['meterId']
print('The list has ' + str(meters.size) + 'meters.')
#print(meters)

The list has 15116meters.
0        CIR0141449180
1        CIR0141600959
2        CIR0141601720
3        CIR0141682188
4        CIR0141441118
5        CIR0501631509
6        CIR0141691244
7        CIR0141692030
9        CIR0141600370
10       CIR0141448720
11       CIR0141456545
12       CIR0141606249
13       CIR0141449251
14       CIR0141601742
15       CIR0141601723
16       CIR0141601476
17       CIR0141448752
18       CIR0141456543
19       CIR0141682187
20       CIR0141682097
             ...      
15186    CIR0141600434
15187    CIR0141689810
15188    CIR0141690537
15189    CIR0141682038
15190    CIR0141600432
15191    CIR0141690595
15192    CIR0141682023
15193    CIR0141690588
15194    CIR0141600418
15195    CIR0141681976
15196    CIR0141690598
15197    CIR0141690593
15198    CIR0141690452
15199    CIR0141690463
15200    CIR0141690171
15201    CIR0141690065
15202    CIR0141682127
15203    CIR0141600439
15204    CIR0141601501
15205    CIR0141601451
Name: meterId, Length: 15116, d

In [ ]:
for min_period in range(360, 721, 30):
    returned_df = run_algorithm(csv_input_filename, min_period)
    number_meters = returned_df[returned_df['complete_periods'] > 0].shape[0]
    print(f"Period of {min_period} days: {number_meters} meters")

In [ ]:
min_date = max(results['start'])
print("Min date: " + str(min_date))

### Try the results

In [ ]:
from datetime import datetime as dt, timedelta as td

In [ ]:
df = pd.read_csv('reactive_values/meter_data_ZIV0046096055_S02.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
expected_delta = td(days=best_time_period)
print(max(df['timestamp']))
print(min(df['timestamp']))

In [ ]:
dt(2021, 3, 11) - dt(2019, 9, 6)

In [ ]:
df['timestamp'].sort_values()

In [ ]:
df = pd.read_csv('time_ranges.csv')

In [ ]:
df.sort_values(by='start')